# Homework 4 - Scalability

## CS109B / Stats 121 / AC209B / CSCI E109B

CS109b Staff

March 8, 2018

 ### Enter which version of the course you are enrolled in: 
 

 <span style="color:blue">
I am enrolled in (e.g. CS109B, AC209)
 </span>

The purpose of this homework is get familiar with scalability and parallelization. 

There are three items to submit:

1. Via Canvas, this populated Jupyter notebook (.ipynb) file
2. Via Canvas, this populated notebook converted to PDF (for inline grading notes)
3. Anonymously share your performance results via this Google form [https://goo.gl/forms/EVrnxj2pzndZvNX22](https://goo.gl/forms/EVrnxj2pzndZvNX22)

## Part 1 - Theory

(a) 30% of your application can be parallelized. It takes 56 seconds to run on your single processor laptop, but only 42 seconds to run on your friend's beefy desktop. How many processors does your friend's machine have? Show your work.


 <span style="color:blue">
answer goes here... 
 </span>

(b) The NVIDIA Telsa K80 in your JupyterHub instances has 4992 cores ([source](http://www.nvidia.com/object/tesla-k80.html)). You have to perform a very large matrix multiplication. Explain what factors would prevent the operation from occurring about 5000 times faster than running the same operation on a single CPU machine. What are the bottlenecks? 

<span style="color:blue">
answer goes here... 
 </span>

## Part 2 - Coding

In this part, we are going to experiment with having code execute in parallel on multiple cores. Let's first determine how many cores a given machine has.

You can run this code on your local machine or on your JupterHub instance. Either is fine.

### 2.1 Setup

In [1]:
from multiprocessing import Process
from multiprocessing import cpu_count
import numpy as np
import time
import random

In [2]:
cpu_count()

8

Every major operating system (MacOS, Windows, Linux) have a means of monitoring CPU utilization across cores.

For MacOS, open `Activity Monitor`, and select CPU usage

For MS-Windows, open `Task Manager`, and look at the performance tab

For Linux, there is the command line `top` command, or for Ubuntu, use `System Monitor`

Verify that the number of cores shown in the appropriate tool corresponds to the number of processors shown above. Leave the CPU monitor window open as it will be used in the subsequent exercises.

**Optional advanced**: your machine may have only half the physical cores listed above. For example, on MacOS, if you go to `Apple` / `About This Mac` / `System Report` only four physical cores could be listed instead of the reported eight. This is due to [hyperthreading](https://en.wikipedia.org/wiki/Hyper-threading). See also [Optimizing Cores](https://macperformanceguide.com/Optimizing-Cores.html)

How many processors are reported above? Does it match what your OS is saying? 

<span style="color:blue">
answer goes here... 
 </span>

### 2.2 Sequential Operations

Here we are going to write some Python that executes in parallel across multiple cores. Specifically, we are going to implement prime factorization in regular Python that operates in parallel across multiple cores.

First, let's look at the sequential version:

In [3]:
from sympy import sieve
MAX_NUM = 1000000
PRIMES = list(sieve.primerange(2,MAX_NUM // 2 + 1))


# the number of composite numbers to factor
N = 100

In [4]:
def calculatePrimeFactors(n):
    primfac = []
    d = 2
    while d*d <= n:
        while (n % d) == 0:
            primfac.append(d)  # supposing you want multiple factors repeated
            n //= d
        d += 1 
    if n > 1:
        primfac.append(n)
    return primfac

In [9]:
print("Starting number crunching")
t0 = time.time()
for i in range(N):
    rand = random.choice(PRIMES) * random.choice(PRIMES) 
    factors = calculatePrimeFactors(rand)
    if (i % (N//10) == 0):
        print(factors)
t1 = time.time()
totalTime = t1 - t0
print("Execution Time: {}".format(totalTime))

Starting number crunching
[117889, 216397]
[134731, 438721]
[274579, 330839]
[138283, 414611]
[91969, 232103]
[122323, 481807]
[277021, 396413]
[12893, 184447]
[152959, 433141]
[11243, 181549]
Execution Time: 3.847149133682251



Now, change the line `N = 100` above so the sequential version takes at least a few seconds to run. Increase by orders of magnitude. You may need a value of `N = 1000` on a recent MacBook for example.

What value of N did you choose and how long did it take to run?

<span style="color:blue">
answer goes here... 
 </span>

### 2.3 Parallel Operations

Let's run in parallel across multiple CPUs.

In [6]:
# the number of concurrent processors to use
nProc = 2

In [10]:
# Now let's run it concurrently
def executeProc():
    # Each process is performing a fraction of the load
    # so it has less numbers to factor
    myN = N // nProc
    for i in range(myN):
        rand = random.choice(PRIMES) * random.choice(PRIMES) 
        factors = calculatePrimeFactors(rand)
        if (i % (myN // 10 * nProc) == 0):
            print(factors)        

In [11]:
t0 = time.time()
procs = []
# Here we create our processes and kick them off
for i in range(nProc):
    proc = Process(target=executeProc, args=())
    procs.append(proc)
    proc.start()
# We use the .join() method in order to wait for
# execution to finish for all of our processes
for proc in procs:
    proc.join()
t1 = time.time()
totalTime = t1 - t0
# we print out the total execution time for our 10
# procs.
print("Execution Time: {}".format(totalTime))

[171917, 436147]
[368773, 417493]
[65867, 252869]
[225781, 320057]
[101267, 475729]
[69233, 406481]
[41023, 131111]
[310361, 374729]
[62143, 224717]
[19489, 148891]
Execution Time: 1.92368483543396


Now, change the line `nProc = 2` above so we use more parallel processors. Increase by 1 or 2 at a time. You may need a value of `nProc = 10` on a recent MacBook for example. Modify the code above to keep track of execution time by number of processes.

What value of nProc produced the fastest time?  

<span style="color:blue">
answer goes here... 
 </span>

### 2.4 - Plotting and Interpretation

(a) Plot the execution time vs number of processes from 1 to nProc + 4 (ie. if nProc == 10 was optimal, plot from 1 to 14 processes). 

In [12]:
# Your code and plot here

(b) According to your OS's CPU monitor, were all cores in use?

<span style="color:blue">
answer goes here... 
 </span>

(c) Why was this optimal based on the number of cores that you have? Why does a higher number of processes not speed things up? How many physical cores do you suspect your machine has vs the number reported by `cpu_count()` in 2.1?

<span style="color:blue">
answer goes here... 
 </span>

### Part 3 - AC209 Problem

Implement prime factorization using `numpy`.

Hint: the book Numpy Cookbook available through Hollis may help you.

Use your function and the same number of `N` sequential cases as above. Time the result.

* How long did numpy take vs the sequential and parallel versions?
* Was your implementation faster than numpy?
* Was numpy using multiple cores or not?

**For all students:**
Don't forget to post your results to  [https://goo.gl/forms/EVrnxj2pzndZvNX22](https://goo.gl/forms/EVrnxj2pzndZvNX22)